# Exemplo de utilização do Pydantic conforme a aula do vídeo
https://www.youtube.com/watch?v=502XOB0u8OY&t=46s

In [ ]:
# instalando a biblioteca do pydantic
!pip install pydantic==2.6.1

In [8]:
# instalando a biblioteca do pydantic
!pip install pydantic[email]

In [13]:
# instalando a biblioteca do FastAPI que será utilizado para criar api de integração e outros exemplos de validação com pydantic
!pip install --only-binary :all: fastapi[all]

  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached typer-0.15.2-py3-none-any.whl.metadata (15 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached httptools-0.6.4-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
  Using cached watchfiles-1.0.4-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached python_multipart-0.0.20-py3-none-any.whl (24 kB)
Using cached uvicorn-0.34.0-py3-none-any.whl (62 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached httptools-0.6.4-

In [21]:
# Primeira aula
## criando objetos que herdam as funcionalidades do pydantic

In [22]:
# importações necessárias
from enum import auto, IntFlag
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)

# criando um objeto que será a definição das regras a serem aplicadas
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

# criando um objeto que herda as funcionalidades do pydantic
class User(BaseModel):
    # um campo "name" que deverá ser do tipo string e tem como exemplo o nome "Arjan"
    name: str = Field(examples=["Arjan"])
    # um campo email que já herda as propriedades EmailStr do pydantic, que tem um exemplo, uma descrição e o campo de congelamento (indicando que se houver mudança será lançado um erro)
    email: EmailStr = Field(
        examples=["example@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    # um campo de senha que já herda as funcionalidades da propriedade SecretStr do pydantic
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    # propriedade regra, aqui é implementado a regra
    role: Role = Field(default=None, description="The role of the user")


def validate(data: dict[str, Any]) -> None:
    '''
    função de validação ao qual objetos podem ser submetidos
    :param data:
    :return:
    '''
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid")
        for error in e.errors():
            print(error)


def main() -> None:
    '''
    método principal simulando o fluxo de um sistema que cria um usuário qualquer e tenta validar esses dados
    :return:
    '''

    # simulação de um objeto a ser validado e aceito
    good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }

    # simulação de um objeto a ser validado e rejeitado
    bad_data = {"email": "<bad data>", "password": "<bad data>"}

    # validando o objeto
    validate(good_data)
    validate(bad_data)


if __name__ == "__main__":
    main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


# Segunda aula
## fluxo de validação usando decoradores (decorators)

In [23]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

# regras utilizando regex, ou seja, busca textual por padrões a serem encontrados
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

# objeto regra
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# objeto usuário com todos os campos e propriedades
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )


    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        '''
        função responsável por realizar a validação exclusiva para o campo "nome" do objeto usuário
        Caso seja validado é retornado o parâmetro "v"
        Caso não seja validado na regra própria é retornado um ValueError
        :param v:
        :return:
        '''
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        '''
        função de validação de uma regra
        :param v:
        :return:
        '''
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        '''
        função de validação de dados obrigatórios a serem preenchidos
        :param v:
        :return:
        '''
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v


def validate(data: dict[str, Any]) -> None:
    '''
    função de validação das propriedades do objeto usuário
    :param data:
    :return:
    '''
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)


def main() -> None:
    '''
    função que simula o fluxo de criação do objeto e das validações a serem executados
    :return:
    '''

    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

# pegando cada objeto criado para ser validado
    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()


if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

# Terceira aula
## validando dados, regras e retornando objetos dos validadores decorados

In [7]:
# importações necessárias para rodar o código


import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
)

#validadores customizados para nome e senha
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

# classe de regra com enumeradores
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

# objeto usuário com herança do modelo pydantic. O Objeto é composto por campos como nome, password e regra
class User(BaseModel):
    name: str = Field(examples=["Example"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],
        default=0,
        validate_default=True,
    )

    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        '''
        função para validação do campo nome
        :param v:
        :return:
        '''
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        '''
        função para validação de uma regra
        :param v:
        :return:
        '''
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        '''
        função de validação de usuário preenchida e antes de uma outra ação
        :param v:
        :return:
        '''
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        '''
        função de validação dos dados de usuário após a execução de uma determinada ação
        :param v:
        :return:
        '''
        if self.role == Role.Admin and self.name != "Arjan":
            raise ValueError("Only Arjan can be an admin")
        return self

    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        '''
        função para serializar a regra
        :param v:
        :return:
        '''
        return v.name

    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        '''
        função para serializar os dados do usuário
        :param serializer:
        :param info:
        :return:
        '''
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)


def main() -> None:
    '''
    função principal para simular o fluxo de um sistema de validação
    :return:
    '''
    data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
        "role": "Admin",
    }
    user = User.model_validate(data)
    if user:
        print(
            "The serializer that returns a dict:",
            user.model_dump(),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),
            sep="\n",
            end="\n\n",
        )
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),
            sep="\n",
            end="\n\n",
        )
        print("The serializer that encodes all values to a dict:", dict(user), sep="\n")


if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Arjan', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Arjan', 'email': 'example@arjancodes.com'}

The serializer that encodes all values to a dict:
{'name': 'Arjan', 'email': 'example@arjancodes.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


# Aula 4
## usando a integração das validações do pydantic com a biblioteca do fastAPI, facilitando a criação, integração e validação de dados dentro de uma determinada API a ser criada

In [20]:
# importação das bibliotecas necessárias para executar o código

from datetime import datetime
from typing import Optional
from typing import List
from uuid import uuid4

from fastapi import FastAPI
from fastapi.responses import JSONResponse
from fastapi.testclient import TestClient
from pydantic import BaseModel, EmailStr, Field, field_serializer, UUID4

# instanciando fastapi para criar os endpoints necessários
app = FastAPI()


#criando o objeto usuário com as propriedades necessárias
class User(BaseModel):
    model_config = {
        "extra": "forbid",
    }
    __users__ = []
    name: str = Field(..., description="Name of the user")
    email: EmailStr = Field(..., description="Email address of the user")
    friends: List[UUID4] = Field(
        default_factory=list, max_items=500, description="List of friends"
    )
    blocked: list[UUID4] = Field(
        default_factory=list, max_items=500, description="List of blocked users"
    )
    signup_ts: Optional[datetime] = Field(
        default_factory=datetime.now, description="Signup timestamp", kw_only=True
    )
    id: UUID4 = Field(
        default_factory=uuid4, description="Unique identifier", kw_only=True
    )

    @field_serializer("id", when_used="json")
    def serialize_id(self, id: UUID4) -> str:
        '''
        função de serialização do ID
        :param id:
        :return:
        '''
        return str(id)


@app.get("/users", response_model=list[User])
async def get_users() -> list[User]:
    '''
    endpoint para retorno da lista de usuários
    :return:
    '''
    print("get_users")
    return list(User.__users__)


@app.post("/users", response_model=User)
async def create_user(user: User):
    '''
    endpoint para criação de um usuário
    :param user:
    :return:
    '''
    print(user)
    User.__users__.append(user)
    return user


@app.get("/users/{user_id}", response_model=User)
async def get_user(user_id: UUID4) -> User | JSONResponse:
    '''
    endpoint responsavel por consultar um usuário pelo ID e retornar o JSON com os dados do usuário
    :param user_id:
    :return:
    '''
    try:
        print(user_id)
        return next((user for user in User.__users__ if user.id == user_id))
    except StopIteration:
        return JSONResponse(status_code=404, content={"message": "User not found"})


def main() -> None:
    '''
    função principal criado para simular o fluxo de um sistema que consome a respectiva API
    :return:
    '''

    # iniciando o ciclo de teste da API juntamente com as validações do Pydantic
    with TestClient(app) as client:
        for i in range(5):
            # fazendo uma requisição do tipo post passando um objeto usuário para a API
            response = client.post(
                "/users",
                json={"name": f"User {i}", "email": f"example{i}@arjancodes.com"},
            )

            # cada assert verifica o resultado esperado do retorno das funções de validação do pydantic com API utilizada nesta aula

            assert response.status_code == 200
            assert response.json()["name"] == f"User {i}", (
                "The name of the user should be User {i}"
            )
            assert response.json()["id"], "The user should have an id"

            user = User.model_validate(response.json())
            assert str(user.id) == response.json()["id"], "The id should be the same"
            assert user.signup_ts, "The signup timestamp should be set"
            assert user.friends == [], "The friends list should be empty"
            assert user.blocked == [], "The blocked list should be empty"

        response = client.get("/users")
        assert response.status_code == 200, "Response code should be 200"
        assert len(response.json()) == 5, "There should be 5 users"

        response = client.post(
            "/users", json={"name": "User 5", "email": "example5@arjancodes.com"}
        )
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "The name of the user should be User 5"
        )
        assert response.json()["id"], "The user should have an id"

        user = User.model_validate(response.json())
        assert str(user.id) == response.json()["id"], "The id should be the same"
        assert user.signup_ts, "The signup timestamp should be set"
        assert user.friends == [], "The friends list should be empty"
        assert user.blocked == [], "The blocked list should be empty"

        response = client.get(f"/users/{response.json()['id']}")
        assert response.status_code == 200
        assert response.json()["name"] == "User 5", (
            "This should be the newly created user"
        )

        response = client.get(f"/users/{uuid4()}")
        assert response.status_code == 404
        assert response.json()["message"] == "User not found", (
            "We technically should not find this user"
        )

        response = client.post("/users", json={"name": "User 6", "email": "wrong"})
        assert response.status_code == 422, "The email address is should be invalid"


if __name__ == "__main__":
    main()

name='User 0' email='example0@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 24, 944773) id=UUID('1feaf7df-8701-45b0-b6e3-0cf354faee5f')
name='User 1' email='example1@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 24, 949287) id=UUID('388b89ba-a17a-4600-b73c-6272c68f0490')
name='User 2' email='example2@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 24, 950284) id=UUID('54616d5a-921f-40d2-b30f-e60a6862dc53')
name='User 3' email='example3@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 24, 951286) id=UUID('fc1b6e03-f814-4e93-a4ee-475823aa80d7')
name='User 4' email='example4@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 24, 952285) id=UUID('0c0792ca-af11-4631-96c6-d4593420c4ee')
get_users
name='User 5' email='example5@arjancodes.com' friends=[] blocked=[] signup_ts=datetime.datetime(2025, 3, 11, 8, 8, 